## Results 1 - Hate Speech Detection

Let's check where the context really helps

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [10]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets
import glob
import json

train_dataset, dev_dataset, test_dataset = load_datasets(add_body=True)

no_context_evals = []
context_evals = []
full_context_evals = []

for path in glob.glob("../evaluations/non-context*"):
    if "category" in path:
        continue
    with open(path) as f:
        obj = json.load(f)
        obj["file"] = path
        no_context_evals.append(obj)

for path in glob.glob("../evaluations/context*"):
    if "category" in path:
        continue

    with open(path) as f:
        obj = json.load(f)
        obj["file"] = path
        context_evals.append(obj)

for path in glob.glob("../evaluations/title-body_*"):
    if "category" in path:
        continue
    with open(path) as f:
        obj = json.load(f)
        obj["file"] = path
        full_context_evals.append(obj)
print(f"We have {len(full_context_evals)} title and body context evaluations")
print(f"We have {len(context_evals)} title context evaluations")
print(f"We have {len(no_context_evals)} no evaluations")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
We have 15 title and body context evaluations
We have 15 title context evaluations
We have 15 no evaluations


In [12]:
import pandas as pd

metric_columns = ["eval_precision", "eval_recall", "eval_f1", "eval_macro_f1"]

df_full_context_evals = pd.DataFrame([
    {**{"file": evaluation["file"]}, **evaluation["metrics"]} for evaluation in full_context_evals
])

df_context_evals = pd.DataFrame([
    {**{"file": evaluation["file"]}, **evaluation["metrics"]} for evaluation in context_evals
])

df_no_context_evals = pd.DataFrame([
    {**{"file": evaluation["file"]}, **evaluation["metrics"]} for evaluation in no_context_evals
])

full_context_df = pd.DataFrame({
    "full context mean": df_full_context_evals[metric_columns].mean(), 
    "full context std": df_full_context_evals[metric_columns].std()})

context_df = pd.DataFrame({
    "context mean": df_context_evals[metric_columns].mean(), 
    "context std": df_context_evals[metric_columns].std()
})

no_context_df = pd.DataFrame({
    "no context mean": df_no_context_evals[metric_columns].mean(), 
    "no context std": df_no_context_evals[metric_columns].std()})


result_df = pd.concat([full_context_df, context_df, no_context_df], axis=1)

result_df

,full context mean,full context std,context mean,context std,no context mean,no context std
eval_precision,0.738500,0.008403,0.750523,0.011957,0.681722,0.019894
eval_recall,0.615619,0.004930,0.603302,0.010823,0.593063,0.017633
eval_f1,0.671448,0.004108,0.668799,0.007305,0.633848,0.005997
eval_macro_f1,0.807807,0.002472,0.806790,0.004181,0.785066,0.003388


Hay que chequear qué pasa con los FP de no usar contexto